In [183]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import time
from sklearn.model_selection import train_test_split

In [184]:
# from google.colab import files
# uploaded = files.upload()# Get the filename of the uploaded file
# orders = list(uploaded.keys())[0]

# Load the data from Excel file into a DataFrame
data = pd.read_csv("orders.csv")

In [185]:
duplicates = data[data.duplicated('user_id')]

In [186]:
#Ensure that each user ID corresponds to a unique user
data = data.drop_duplicates('user_id')


In [187]:
data['access_datetime'] = pd.to_datetime(data['access_date'] + ' ' + data['access_time'])


In [188]:
# Derive additional time-related features
data['hourly_attendance'] = data['access_datetime'].dt.hour
data['daily_attendance'] = data['access_datetime'].dt.date

In [189]:

# Calculate total visits per day
total_visits_per_day = data.groupby('daily_attendance').size()

# Calculate total visitors per day
total_visitors_per_day = data.groupby('daily_attendance')['user_id'].nunique()

# Calculate peak visiting hours
peak_visiting_hours = data.groupby('hourly_attendance').size().idxmax()

In [190]:
start_time = time.time()


In [191]:

# Aggregate hourly attendance data to obtain total visits per day
total_visits_per_day = data.groupby('access_date')['hourly_attendance'].sum()

# Merge aggregated data with the original DataFrame
data = data.merge(total_visits_per_day, on='access_date', suffixes=('', '_total'))

# Define features (X) and target variable (y)
X = data[['access_date']]  # Features
y = data['hourly_attendance_total']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [192]:
# Record the end time
end_time = time.time()

# Calculate the time taken
train_time = end_time - start_time
print("Training time:", train_time, "seconds")

Training time: 0.04449915885925293 seconds


In [193]:
# Concatenate the training and testing data before applying one-hot encoding
combined_data = pd.concat([X_train, X_test])
combined_data_encoded = pd.get_dummies(combined_data)

# Split the combined encoded data back into training and testing sets
X_train_encoded = combined_data_encoded[:len(X_train)]
X_test_encoded = combined_data_encoded[len(X_train):]

In [194]:

# Concatenate the training and testing data before applying one-hot encoding
combined_data = pd.concat([X_train, X_test])
combined_data_encoded = pd.get_dummies(combined_data)

# Split the combined encoded data back into training and testing sets
X_train_encoded = combined_data_encoded[:len(X_train)]
X_test_encoded = combined_data_encoded[len(X_train):]

# Initialize and train the Linear Regression model
model = RandomForestRegressor(random_state=42)
model.fit(X_train_encoded, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test_encoded)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R^2 Score:", r2)
from random import randint
d = pd.DataFrame()
d["access_date_2023-04-01"] = [randint(0,100) for i in range(10)]
d["access_date_2023-04-03"] = [randint(0,100) for i in range(10)]
d["access_date_2023-04-04"] = [randint(0,100) for i in range(10)]
print(d)
print(model.predict(d))


RMSE: 3.5834341071101057
R^2 Score: 0.737938775510204
   access_date_2023-04-01  access_date_2023-04-03  access_date_2023-04-04
0                      32                       6                      37
1                      53                      44                       7
2                      95                      67                      45
3                      49                      71                      19
4                      46                      44                      93
5                      92                      64                      10
6                      19                      25                      22
7                      65                      85                      87
8                      93                     100                      16
9                      76                      15                      75


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- access_date_2023-06-08
- access_date_2023-08-03
- access_date_2023-08-14
- access_date_2023-08-20
- access_date_2023-11-04


In [ ]:
# save the trained model
import pickle
filename ='RFG.sav'
pickle.dump(model, open(filename,'wb'))